# Unsupervised Learning

## Gensim LDA (Latent Dirichlet Allocation)

In [1]:
import numpy as np
import pandas as pd

import sys
import os

import gensim
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from gensim.matutils import corpus2csc

from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

import tqdm

import tomotopy as tp

from preprocess import preprocess, tokenizer

/home/firewrz/PycharmProjects/COVID-Vaccine-Tweet/venv/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
data_root = '../data/cvt/'
df = pd.read_csv(os.path.join(data_root, 'covidvaccine.csv'))
df

/home/firewrz/PycharmProjects/COVID-Vaccine-Tweet/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,hashtag_count
0,Ashwin,India 🇮🇳,I'm Just A Man On a Mission For My Vision!\n\n...,2018-01-08 06:01:00,89.0,3108.0,6780.0,0,2020-08-09 19:15:00,Huge Thanks And Best Wishes With @SerumInstInd...,covisheild,Twitter for Android,1
1,K. Kartik Rao,Mumbai and Nagpur India,Actor 🎭 \n\nhttps://t.co/u7GZfoWBDt\n\nSubscri...,2016-12-10 16:06:00,225.0,257.0,25362.0,0,2020-08-09 20:02:00,Any update of #CovidVaccine ????,covidvaccine,Twitter for Android,1
2,Mac Patrick (Road Hater) Henry,German Bundesrepublik,Liberty lover. \nServing the people of the 🇺🇸....,2014-01-11 19:47:00,338.0,277.0,28305.0,0,2020-08-09 21:21:00,From a private frontline COVID nurses group to...,covid19 covidvaccinerace covidvaccine pandemic,Twitter Web App,4
3,lydia Bell,Earth,"Mummy, animal lover , energy worker , awake in...",2009-03-23 01:09:00,2038.0,1105.0,63422.0,0,2020-08-09 21:29:00,So you want the whole world to take a #covidva...,covidvaccine,Twitter for iPhone,1
4,Lou Ann Donovan,"Kansas, USA",Live Young Look Young BE Young! Certified Inte...,2009-03-17 05:42:00,4365.0,4581.0,2505.0,0,2020-08-09 22:55:00,Watch to learn about the Phase 3 clinical tria...,covid19 covidvaccine covidclinicaltrial,Twitter for iPhone,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154533,Script Your Future '21,DMV,HowardU and Script Your Future has joined forc...,2021-01-31 18:40:41,15.0,42.0,4.0,0,2021-02-28 23:55:25,* Continue with routine preventive measures (s...,NaN,Twitter Web App,0
154534,eastvillagetwt,"East Village, New York, NY","Politics, the environment, science, culture, t...",2011-08-27 19:24:17,1968.0,5001.0,136110.0,0,2021-02-28 23:56:37,COVID: California Suspends System Of Shared Va...,covid19 covidvaccine,Twitter for iPhone,2
154535,Yogita,"New York, NY",Primum non nocere 💕👩🏽‍⚕️ | Tweets are my own. ...,2015-08-07 01:48:57,17.0,58.0,1194.0,0,2021-02-28 23:57:00,Today I got my first dose of the #COVIDVaccine...,covidvaccine,Twitter for Android,1
154536,DrZ,USA,NaN,2016-03-23 13:00:36,54.0,4.0,4.0,0,2021-02-28 23:57:06,Here's Where to Receive a COVID Vaccine in Ill...,illinois covidvaccine,Buffer,2


In [3]:
df['text'] = df['text'].apply(lambda x: preprocess(x))

df

/home/firewrz/PycharmProjects/COVID-Vaccine-Tweet/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,hashtag_count
0,Ashwin,India 🇮🇳,I'm Just A Man On a Mission For My Vision!\n\n...,2018-01-08 06:01:00,89.0,3108.0,6780.0,0,2020-08-09 19:15:00,huge thanks best wishes seruminstindia adarpoo...,covisheild,Twitter for Android,1
1,K. Kartik Rao,Mumbai and Nagpur India,Actor 🎭 \n\nhttps://t.co/u7GZfoWBDt\n\nSubscri...,2016-12-10 16:06:00,225.0,257.0,25362.0,0,2020-08-09 20:02:00,update covidvaccine,covidvaccine,Twitter for Android,1
2,Mac Patrick (Road Hater) Henry,German Bundesrepublik,Liberty lover. \nServing the people of the 🇺🇸....,2014-01-11 19:47:00,338.0,277.0,28305.0,0,2020-08-09 21:21:00,private frontline covid nurses group today cov...,covid19 covidvaccinerace covidvaccine pandemic,Twitter Web App,4
3,lydia Bell,Earth,"Mummy, animal lover , energy worker , awake in...",2009-03-23 01:09:00,2038.0,1105.0,63422.0,0,2020-08-09 21:29:00,want whole world take covidvaccine s fast trac...,covidvaccine,Twitter for iPhone,1
4,Lou Ann Donovan,"Kansas, USA",Live Young Look Young BE Young! Certified Inte...,2009-03-17 05:42:00,4365.0,4581.0,2505.0,0,2020-08-09 22:55:00,watch learn phase clinical trial covid vaccine...,covid19 covidvaccine covidclinicaltrial,Twitter for iPhone,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154533,Script Your Future '21,DMV,HowardU and Script Your Future has joined forc...,2021-01-31 18:40:41,15.0,42.0,4.0,0,2021-02-28 23:55:25,continue routine preventive measures vaccinati...,NaN,Twitter Web App,0
154534,eastvillagetwt,"East Village, New York, NY","Politics, the environment, science, culture, t...",2011-08-27 19:24:17,1968.0,5001.0,136110.0,0,2021-02-28 23:56:37,covid california suspends system shared vaccin...,covid19 covidvaccine,Twitter for iPhone,2
154535,Yogita,"New York, NY",Primum non nocere 💕👩🏽‍⚕️ | Tweets are my own. ...,2015-08-07 01:48:57,17.0,58.0,1194.0,0,2021-02-28 23:57:00,today got first dose covidvaccine super thankful,covidvaccine,Twitter for Android,1
154536,DrZ,USA,NaN,2016-03-23 13:00:36,54.0,4.0,4.0,0,2021-02-28 23:57:06,heres receive covid vaccine illinois illinois ...,illinois covidvaccine,Buffer,2


In [4]:
df['text']

/home/firewrz/PycharmProjects/COVID-Vaccine-Tweet/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0         huge thanks best wishes seruminstindia adarpoo...
1                                       update covidvaccine
2         private frontline covid nurses group today cov...
3         want whole world take covidvaccine s fast trac...
4         watch learn phase clinical trial covid vaccine...
                                ...                        
154533    continue routine preventive measures vaccinati...
154534    covid california suspends system shared vaccin...
154535     today got first dose covidvaccine super thankful
154536    heres receive covid vaccine illinois illinois ...
154537    utah slc sold walmart vaccine first dose book ...
Name: text, Length: 154538, dtype: object

In [4]:
# Prepare corpus
import nltk
from nltk.corpus import stopwords
def get_tp_corpus(docs):
    porter_stemmer = nltk.PorterStemmer().stem
    english_stops = set(porter_stemmer(w) for w in stopwords.words('english'))
    tp_corpus = tp.utils.Corpus(
        tokenizer=tp.utils.SimpleTokenizer(porter_stemmer), 
        stopwords=lambda x: x in english_stops
    )
    tp_corpus.process(d.lower() for d in docs)
    return tp_corpus
tp_corpus = get_tp_corpus(list(df['text']))

/home/firewrz/PycharmProjects/COVID-Vaccine-Tweet/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
def compute_coherence_values_tp(tp_corpus, k, a, b):
    lda_model = tp.LDAModel(k=k,alpha=a,eta=b,corpus=tp_corpus)
    lda_model.train(500)
    co_mass = tp.coherence.Coherence(lda_model,coherence="u_mass")
    co_cv = tp.coherence.Coherence(lda_model,coherence="c_v")
    return co_mass.get_score(), co_cv.get_score()

/home/firewrz/PycharmProjects/COVID-Vaccine-Tweet/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
already_run = False

# Topics range (length = 10)
min_topics = 2
max_topics = 22
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter (length = 5)
alpha = list(np.arange(0.01, 1, 0.1))

# Beta parameter (length = 5)
beta = list(np.arange(0.01, 1, 0.1))

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence_cv': [],
                 "Coherence_umass":[],
                }

if not already_run:
    # Can take a long time to run
    pbar = tqdm.tqdm(total=2200)
    
    # Iterate through number of topics
    for k in topics_range:
        # Iterate through alpha values
        for a in alpha:
            # Iterate through beta values
            for b in beta:
                # Get the coherence score for the given parameters
                umass,cv = compute_coherence_values_tp(tp_corpus=tp_corpus,k=k, a=a, b=b)
                #print(umass,cv)
                # Save the model results
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence_cv'].append(cv)
                model_results['Coherence_umass'].append(umass)


                pbar.update(1)
                coherence_results = pd.DataFrame(model_results)
                coherence_results.to_csv(os.path.join('lda_tuning_results.csv'), index=False)
    pbar.close()
else:
    coherence_results = pd.read_csv(os.path.join('lda_tuning_results.csv'))

/home/firewrz/PycharmProjects/COVID-Vaccine-Tweet/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  0%|          | 1/2200 [00:15<9:45:04, 15.96s/it]

KeyboardInterrupt: 

In [6]:
mdl = tp.LDAModel(k=20,corpus=tp_corpus,alpha=0.91,eta=0.91)
mdl.train(500)

/home/firewrz/PycharmProjects/COVID-Vaccine-Tweet/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
mdl.summary()

topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq

prepared_data = pyLDAvis.prepare(
    topic_term_dists, 
    doc_topic_dists, 
    doc_lengths, 
    vocab, 
    term_frequency
)
pyLDAvis.save_html(prepared_data, 'ldavis.html')

/home/firewrz/PycharmProjects/COVID-Vaccine-Tweet/venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<Basic Info>
| LDAModel (current version: 0.11.1)
| 154535 docs, 1531044 words
| Total Vocabs: 76786, Used Vocabs: 76786
| Entropy of words: -7.72703
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.14832
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 20 (the number of topics between 1 ~ 32767)
| alpha: [0.91] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.91 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 2213102021 (random seed)
| trained in version 0.11.1
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [2.55105551e-04 1.24684768e-04 2.30085403e-01